In [1]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time
import pickle

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import sys
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import *
from metpy.calc import lat_lon_grid_deltas
from pioneer.das.api.platform import Platform
from scipy.spatial.transform import Rotation as R
from pioneer.das.api.egomotion.imu_egomotion_provider import IMUEgomotionProvider as emp 
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

2023-04-24 16:03:07.656185: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 16:03:07.746987: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-24 16:03:08.145452: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/derm/anaconda3/envs/py39/lib/python3.9/site-packages/cv2/../../lib64:
2023-04-24 16:03:08.145505: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_p

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-04-24 16:03:08.711164: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 16:03:08.711926: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 16:03:08.712137: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 16:03:08.712443: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


# Goal: Estimate acceleration profile $\hat{m}$ Using Newton-Raphson to compensate for distortion between similar point cloud

Through the first order Taylor approximation we define: 

$\mathbf{h}(\hat{m})$ as the means of voxel centers in cloud $j$ passed through the distortion operator $\mathbb{C}$ according to estimated motion profile $\hat{m}$. 

$\mathbf{H}$ is the Jacobian of $\mathbf{h}$, which is multiplied by a linear correction factor $\delta m$.

$    y_i = \mathbf{h}(\hat{m}) + \mathbf{H} \delta m $


In [2]:
# load point cloud
# # no distortion
# old_cloud =  np.load("sample_data/paper_figures/case1/raw_frame_0.npy")
# # m_hat = np.array([0., 0., 0., 0., 0., 0.0]) #actual motion
# m_hat = np.array([-3., 0., 0., 0., 0., 0.1]) #test wrap around

# # movement in x
# old_cloud =  np.load("sample_data/paper_figures/case2/raw_frame_3.npy") 
# m_hat = np.array([3, 0, 0., 0., 0., 0])
# # m_hat = np.array([3, 0, 0., 0., 0., -0.2]) #FOR DEBUG-- deform just a little
# gt =  np.load("sample_data/paper_figures/case2/base_vel_2.npy")

# movement in x, y, & yaw
old_cloud =  np.load("sample_data/paper_figures/case3/raw_frame_1.npy") 
m_hat = np.array([3, -1, 0., 0., 0., -1])
# m_hat = np.array([3, -1, 0.1, 0.1, 0.05, -1]) #FOR DEBUG-- deform a little extra
gt =  np.load("sample_data/paper_figures/case3/base_vel_2.npy")
# print(gt) 

# period_lidar = 1
# t_scale = (2*np.pi)/(-m_hat[-1] + (2*np.pi/period_lidar))
# print(t_scale)
# m_hat = m_hat*t_scale
# # m_hat[-1] = m_hat[-1]*t_scaled
# print(m_hat)


#downsample
old_cloud = old_cloud[::5,:]

In [3]:
from remove_motion_basic import linear_correction_old as lc 

new_cloud = lc(old_cloud, m_hat)
# undistorted = lc(new_cloud, -m_hat)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
# disp.append(Points(undistorted, c = "#CB2314"))
# disp.append(Points(old_cloud, c = "#CB2314"))
disp.append(Points(new_cloud, c = "#3F5151"))
plt.show(disp, "test viz")
ViewInteractiveWidget(plt.window)

/home/derm/anaconda3/envs/py39/lib/python3.9/site-packages/vtkmodules/util/numpy_support.py:74: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _vtk_np = {vtkConstants.VTK_BIT:numpy.bool,


ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

# Analytically calculating derivatives with SymPy

In [5]:
from sympy import *

x, y, z = symbols('x y z')

init_printing(use_unicode=True)

In [6]:
#sovles for the derivative
ans1 = diff(cos(x), x)
print(ans1)

#creates an unevaluated derivative
ans2 = Derivative(cos(x), x)
print(ans2)

-sin(x)
Derivative(cos(x), x)


In [7]:
ans3 = integrate(sin((x+1)/2)*(cos(x)-sin(x))**2, x)
# print(latex(ans3)) #-> converts to LaTeX so I don't have to re-copy everything 

integrate(sin((x+1)/2)*(cos(x)-sin(x))**2, x)


In [8]:
a = Matrix([[10*x, 2], 
            [3, 4*x]])
# A = integrate(a**2)
# print(latex(A))
integrate(a**2)

⎡     3                   ⎤
⎢100⋅x                2   ⎥
⎢────── + 6⋅x     14⋅x    ⎥
⎢  3                      ⎥
⎢                         ⎥
⎢                  3      ⎥
⎢       2      16⋅x       ⎥
⎢   21⋅x       ───── + 6⋅x⎥
⎣                3        ⎦

In [9]:
b = Matrix([[x + 1, 2*y],
            [3*x**2*y, 3 + x + y**2]])
diff(b, y, x)

⎡ 0   0⎤
⎢      ⎥
⎣6⋅x  0⎦

# Derive Jacobain H

### $\mathbf{H} \in \mathbb{R}^{4N \times 6} $

### $\mathbf{H} = 
\begin{bmatrix}
^{(n)}H_{x} ~ ^{(n)}H_{y} ~ ^{(n)}H_{z} ~ ^{(n)}H_{\phi} ~ ^{(n)}H_{\theta} ~ ^{(n)}H_{\psi}
\end{bmatrix}$

### $ ^{(n)}\mathbf{H}_x =  \frac{\delta{\mathbb{C}}}{\delta x} ~ ^{(n)}y_j \in \mathbb{R}^{4 \times 1}$

### Here $^{(n)}y_j$ refers to distribution mean from scan j inside voxel n, denoted in homogenous coordinates

$[x, y, z] \rightarrow [x, y, z, 1] $ 

<!-- <span style="color:red"> TODO: figure out if I need to use homogeneous coordinates for $y_j$ </span> -->

# Calculate correction  function $\mathbf{C}$ 

## $\mathbf{C}(\hat{m}) = T_{\text{rect}}(M(\hat{m})) \in \mathbb{R}^{4 \times 4} $

Here, the rectifying transformation $T_{\text{rect}}$ for each point in $y_j$ depends on scaled values of $\hat{m}$ which are concatenated to form matrix $M$. Since points are stored in the order they are recieved and are somewhat even spread throughout the scene, we can approximate $M$ through a simple linspace matrix without needing to transform to a spherical representation. $t_{\text{scale}}$ is a scaling parameter to normalize the turn rate of the sensor in the presence of composite yaw rotation.

## $
M = \bigg{(} \{ r: r = t_{\text{scale}} \frac{p}{P}, p \in \{ 0, 1, ..., P \} \} \bigg{)} \hat{m}
\in \mathbb{R}^{P \times 6}
% \begin{bmatrix}
% 1 & 1 & 1 & 1 
% \end{bmatrix}
$

## $t_{\text{scale}} = \frac{2\pi}{-\psi + \omega_{\text{LIDAR}} } $ 


My old way of "unwinding" was acomplished by converting the point cloud to spherical coordinates, rescaling the yaw angle, and then converting back to Cartesian 

<!-- ## $y_{j, \text{unwound}} =
\text{s2c} \bigg{(} \text{c2s}(y_{j})
\begin{bmatrix}
    1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & \psi_{\text{rect}}
\end{bmatrix} \bigg{)}$
 -->
Unfortunately, c2s() is a nonlinear operation so it makes things a little tricky when we need to do everything via linear algebra

<!-- ## $ \text{c2s}(y_j) = 
\begin{bmatrix}
x_1 & y_1 & z_1 \\
x_2 & y_2 & z_2 \\
 & \vdots &
\end{bmatrix}
\rightarrow
\begin{bmatrix}
r_1 & \phi_1 & \theta_1 \\
r_2 & \phi_2 & \theta_2 \\
r_3 & \phi_3 & \theta_3
\end{bmatrix}
$
 -->
<!-- ### GPT:
lol it really got this wro

$ ^{(n)}y_j = \begin{bmatrix}
x \\
y \\
z
\end{bmatrix}
$

$ B = \begin{bmatrix}
\text{sin}(\theta) ~ \text{cos}(\phi) & \text{cos}(\theta) ~ \text{cos}(\phi) & -\text{sin}(\phi) \\
\text{sin}(\theta)\text{sin}(\phi) & \text{cos}(\theta)\text{sin}(\phi) & \text{cos}(\phi) \\
\text{cos}(\theta) & -\text{sin}(\theta) & 0
\end{bmatrix}
$

$B ~ ^{(n)}y_j = ^{(n)}y_{j, \text{spherical}} $ -->


<!-- Putting everything together:

 $
\mathbb{C}(y_j, \hat{m}) =
M ~ T_{\text{rect}} \bigg{[} \text{s2c} \bigg{(} \text{c2s}(y_{j})
\begin{bmatrix}
    1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & \psi_{\text{rect}}
\end{bmatrix} \bigg{)},
\begin{bmatrix}
    1 \\
    \vdots 
\end{bmatrix}
\bigg{]}^T$
 -->

In [10]:
period_lidar = 1
m_hat = np.array([3, 0, 0., 0., 0., -0.5])
t_scale = (2*np.pi)/(-m_hat[-1] + (2*np.pi/period_lidar))
# print(t_scale)
lsvec = np.linspace(0,t_scale, 100)

print(np.shape(m_hat))
M = m_hat * np.array([lsvec, lsvec, lsvec, lsvec, lsvec, lsvec]).T
# print(np.shape(M))
# print(M)

(6,)


## Get transform $T_{\text{rect}}$

In [11]:
x, y, z, phi, theta, psi = symbols('x y z phi theta psi')
# m_hat = Matrix([x, y, z, phi, theta, phi]).T
m_hat = np.array([1,2,3,0.4,0.5,0.6])
# print( "m_hat = ", pretty(m_hat))

T_roll = Matrix([[1, 0, 0, 0],
                 [0, cos(phi), sin(phi), 0],
                 [0, -sin(phi), cos(phi), 0],
                 [0, 0, 0, 1]])
# print("\n T_roll =") 
# print(pretty(T_roll))

T_pitch = Matrix([[cos(theta), 0, -sin(theta), 0],
                 [0, 1, 0, 0],
                 [sin(theta), 0, cos(theta), 0],
                 [0, 0, 0, 1]])
# print("\n T_pitch =") 
# print(pretty(T_pitch))

#Recall we are setting T_yaw to identity here since scan axis-aligned motion is already compensated for
T_yaw = Matrix([[cos(psi), sin(psi), 0, 0],
                 [-sin(psi), cos(psi), 0, 0],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
# print("\n T_yaw =") 
# print(pretty(T_yaw))

T_trans = Matrix([[1, 0, 0, x],
                 [0, 1, 0, y],
                 [0, 0, 1, z],
                 [0, 0, 0, 1]])
# print("\n T_trans =") 
# print(pretty(T_trans))

T_rect = (T_roll * T_pitch * T_yaw * T_trans)
# T_rect = (T_roll * T_pitch * T_yaw * T_trans).inv()
print("T_rect =")
T_rect

T_rect =


⎡           cos(ψ)⋅cos(θ)                          sin(ψ)⋅cos(θ)                 -sin(θ)                            
⎢                                                                                                                   
⎢sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)  sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)   sin(φ)⋅cos(θ)  x⋅(sin(φ)⋅sin(θ)⋅cos(ψ)
⎢                                                                                                                   
⎢sin(φ)⋅sin(ψ) + sin(θ)⋅cos(φ)⋅cos(ψ)  -sin(φ)⋅cos(ψ) + sin(ψ)⋅sin(θ)⋅cos(φ)  cos(φ)⋅cos(θ)  x⋅(sin(φ)⋅sin(ψ) + sin(
⎢                                                                                                                   
⎣                 0                                      0                          0                               

      x⋅cos(ψ)⋅cos(θ) + y⋅sin(ψ)⋅cos(θ) - z⋅sin(θ)                             ⎤
                                                                               ⎥
 - sin(ψ)⋅cos(φ)) 

In [12]:
#try simplifying T_rect before takiking derivatives!
T_rect = simplify(T_rect)
T_rect
# print(latex(T_rect))
# print(T_rect)

⎡           cos(ψ)⋅cos(θ)                          sin(ψ)⋅cos(θ)                 -sin(θ)                            
⎢                                                                                                                   
⎢sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)  sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)   sin(φ)⋅cos(θ)  x⋅(sin(φ)⋅sin(θ)⋅cos(ψ)
⎢                                                                                                                   
⎢sin(φ)⋅sin(ψ) + sin(θ)⋅cos(φ)⋅cos(ψ)  -sin(φ)⋅cos(ψ) + sin(ψ)⋅sin(θ)⋅cos(φ)  cos(φ)⋅cos(θ)  x⋅(sin(φ)⋅sin(ψ) + sin(
⎢                                                                                                                   
⎣                 0                                      0                          0                               

     x⋅cos(ψ)⋅cos(θ) + y⋅sin(ψ)⋅cos(θ) - z⋅sin(θ)                             ⎤
                                                                              ⎥
 - sin(ψ)⋅cos(φ)) + 

In [13]:
# print(diff(T_rect, x))
# diff(T_rect,y)
# print(diff(T_rect, z))
# print(diff(T_rect, psi))
# diff(T_rect, phi)
diff(T_rect, theta)
# print(diff(T_rect, theta))
# print(latex(diff(T_rect, phi)))

⎡   -sin(θ)⋅cos(ψ)        -sin(ψ)⋅sin(θ)        -cos(θ)                -x⋅sin(θ)⋅cos(ψ) - y⋅sin(ψ)⋅sin(θ) - z⋅cos(θ)
⎢                                                                                                                   
⎢sin(φ)⋅cos(ψ)⋅cos(θ)  sin(φ)⋅sin(ψ)⋅cos(θ)  -sin(φ)⋅sin(θ)  x⋅sin(φ)⋅cos(ψ)⋅cos(θ) + y⋅sin(φ)⋅sin(ψ)⋅cos(θ) - z⋅sin
⎢                                                                                                                   
⎢cos(φ)⋅cos(ψ)⋅cos(θ)  sin(ψ)⋅cos(φ)⋅cos(θ)  -sin(θ)⋅cos(φ)  x⋅cos(φ)⋅cos(ψ)⋅cos(θ) + y⋅sin(ψ)⋅cos(φ)⋅cos(θ) - z⋅sin
⎢                                                                                                                   
⎣         0                     0                  0                                         0                      

          ⎤
          ⎥
(φ)⋅sin(θ)⎥
          ⎥
(θ)⋅cos(φ)⎥
          ⎥
          ⎦

In [14]:
# # print(T_rect) # -> take printed output and convert to numpy array

# # from tensorflow.math import sin, cos
# from numpy import sin, cos

# x = np.array([1,2,3])#[:,None]
# y = np.array([1,2,3])#[:,None]
# z = np.array([1,2,3])#[:,None]
# phi = np.array([1,2,3])#[:,None]
# theta = np.array([1,2,3])#[:,None]
# psi = np.array([1,2,3])#[:,None]
# test = np.array([cos(x) * cos(y) / sin(phi), cos(x) * cos(y) / sin(phi)])
# print(np.shape(test))
# print(test)

# T_rect_np = np.array([[cos(psi)*cos(theta)/(sin(psi)**2*sin(theta)**2 + sin(psi)**2*cos(theta)**2 + sin(theta)**2*cos(psi)**2 + cos(psi)**2*cos(theta)**2), (sin(phi)*sin(theta)*cos(psi) - sin(psi)*sin(theta)**2*cos(phi) - sin(psi)*cos(phi)*cos(theta)**2)/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), (sin(phi)*sin(psi)*sin(theta)**2 + sin(phi)*sin(psi)*cos(theta)**2 + sin(theta)*cos(phi)*cos(psi))/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), -x], [sin(psi)*cos(theta)/(sin(psi)**2*sin(theta)**2 + sin(psi)**2*cos(theta)**2 + sin(theta)**2*cos(psi)**2 + cos(psi)**2*cos(theta)**2), (sin(phi)*sin(psi)*sin(theta) + sin(theta)**2*cos(phi)*cos(psi) + cos(phi)*cos(psi)*cos(theta)**2)/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), (-sin(phi)*sin(theta)**2*cos(psi) - sin(phi)*cos(psi)*cos(theta)**2 + sin(psi)*sin(theta)*cos(phi))/(sin(phi)**2*sin(psi)**2*sin(theta)**2 + sin(phi)**2*sin(psi)**2*cos(theta)**2 + sin(phi)**2*sin(theta)**2*cos(psi)**2 + sin(phi)**2*cos(psi)**2*cos(theta)**2 + sin(psi)**2*sin(theta)**2*cos(phi)**2 + sin(psi)**2*cos(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2*cos(psi)**2 + cos(phi)**2*cos(psi)**2*cos(theta)**2), -y], [-sin(theta)/(sin(theta)**2 + cos(theta)**2), sin(phi)*cos(theta)/(sin(phi)**2*sin(theta)**2 + sin(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2 + cos(phi)**2*cos(theta)**2), cos(phi)*cos(theta)/(sin(phi)**2*sin(theta)**2 + sin(phi)**2*cos(theta)**2 + sin(theta)**2*cos(phi)**2 + cos(phi)**2*cos(theta)**2), -z], 
#                       [np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.ones(len(x))]])
# print(np.shape(T_rect_np))
# test = np.transpose(T_rect_np, (2,0,1))
# print(test)

In [15]:
# y_j = y_j[:3]
# if np.shape(y_j)[1] == 3:
#     y_j = np.append(y_j, np.ones([len(y_j),1]), axis = 1)

# # y_j_tf = tf.convert_to_tensor(y_j[:,:,None])
# # test_tf = tf.convert_to_tensor(test)
# # print(np.shape(y_j_tf))
# # print(np.shape(test_tf))

# # ans = y_j_tf * test_tf

# print(np.shape(y_j[:,:,None]))
# print(np.shape(test))
# # ans = y_j[:,:,None] * test
# ans = test @ y_j[:,:,None]


# print(np.shape(ans))
# print(ans)
# ans = np.reshape(ans, (-1, 1))
# print(ans)

In [16]:
# diff(T_rect, x)
diff(T_rect, phi)

# dT_rect_dx = diff(T_rect, x)
# pprint(dT_rect_dx)

⎡                  0                                      0                          0                              
⎢                                                                                                                   
⎢sin(φ)⋅sin(ψ) + sin(θ)⋅cos(φ)⋅cos(ψ)   -sin(φ)⋅cos(ψ) + sin(ψ)⋅sin(θ)⋅cos(φ)  cos(φ)⋅cos(θ)   x⋅(sin(φ)⋅sin(ψ) + si
⎢                                                                                                                   
⎢-sin(φ)⋅sin(θ)⋅cos(ψ) + sin(ψ)⋅cos(φ)  -sin(φ)⋅sin(ψ)⋅sin(θ) - cos(φ)⋅cos(ψ)  -sin(φ)⋅cos(θ)  x⋅(-sin(φ)⋅sin(θ)⋅cos
⎢                                                                                                                   
⎣                  0                                      0                          0                              

                             0                                                   ⎤
                                                                                 ⎥
n(θ)⋅cos(φ)⋅co

In [17]:
#substitue in values
expr = diff(T_rect, phi)
# print(expr)
# expr = T_rect
# print(m_hat)
ans = expr.subs([(x, m_hat[0]), 
                 (y, m_hat[1]),
                 (z, m_hat[2]),
                 (phi, m_hat[3]),
                 (theta, m_hat[4]),
                 (psi, m_hat[5])])
print(type(np.array(ans)))
print("\n", np.array(ans).astype(np.float64))

<class 'numpy.ndarray'>

 [[ 0.          0.          0.          0.        ]
 [ 0.58433397 -0.07206591  0.80830707  2.86512335]
 [ 0.36598239 -0.86560155 -0.34174675 -2.39046095]
 [ 0.          0.          0.          0.        ]]


# Test rectifying motion distortion usuing SymPy Linalg technique

In [18]:
from remove_motion_basic import get_H
from remove_motion_basic import linear_correction_old as lc 

m_hat = np.array([3, 0, 0, 0, 0, 0])
skip = 1
y_j = old_cloud[::skip]
# y_j = old_cloud[::skip] + np.array([0.5, 0.5, 0.]) #test offset - hurts performance but doesn't cause anything to explode
y_i = new_cloud[::skip] #has distortion already corrected

# #remove ground plane
# y_i = y_i[y_i[:,2] > -1]
# y_j = y_j[y_j[:,2] > -1]

H = get_H(y_j = y_j, m_hat = m_hat )

#plot result
plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(y_i, c = "#CB2314", r=3.5))
disp.append(Points(y_j, c = "#3F5151", r=3.5))
plt.show(disp, "test viz")
ViewInteractiveWidget(plt.window)

/home/derm/anaconda3/envs/py39/lib/python3.9/site-packages/vtkmodules/util/numpy_support.py:74: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _vtk_np = {vtkConstants.VTK_BIT:numpy.bool,


ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [19]:
print(np.shape(H))
print(np.shape(y_j)[0]*4)

(66256, 6)
66256


# Simple Test using pre-rectified point cloud
#### in this example we don't need to worry about iteratively finding correspondences
(y_i and y_j are the same length and are ordered accordingly) 

In [20]:
#assume initial correction factor of 0 for each solution component
m_hat = np.array([0., 0, 0, 0, 0, 0])
skip = 1 #50
y_i = new_cloud[::skip] #baseline
y_j = old_cloud[::skip] #distorted cloud
# y_j = old_cloud[::skip] + np.array([0.5, 0.5, 0.]) #test offset - hurts performance but doesn't cause anything to explode

# #add noise 
# noise_scale = 0.01
# y_i += np.random.randn(np.shape(y_i)[0],np.shape(y_i)[1])*noise_scale 
# y_j += np.random.randn(np.shape(y_j)[0],np.shape(y_j)[1])*noise_scale 

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(y_i[:,:3], c = "#2c7c94 ", alpha = 1, r=3.5))

runlen = 10
for count in range(runlen):
    
    print("~~~~ iteration ", count, "~~~~~~~~~~~")
    print("m_hat = ", m_hat)
    
    #apply last estimate of correction
    y_j_undistort = lc(y_j, m_hat)
    print(np.shape(y_j_undistort))
    
    #delta_m =  ((H^T*H)^-1)(H^T)(yi - yj*)
    H = get_H(y_j_undistort, m_hat)
    residual = (np.append(y_i, np.ones([len(y_i),1]), axis = 1) -
                np.append(y_j_undistort, np.ones([len(y_i),1]), axis = 1)).flatten()
    print("residual", np.shape(residual))
    delta_m = np.linalg.pinv(H.T @ H) @ H.T @ residual
    m_hat[:3] -= delta_m[:3]
    m_hat[3:] += delta_m[3:] #TODO: figure out why angles are backwards...    
    
    #plot updated cloud2
#     color = [0.5 + count/(runlen*2), 1 - count/runlen, count/runlen]
    disp.append(Points(y_j_undistort[:,:3], c = "#a65852 ", alpha = (count+1)/(runlen+1), r=3.5))

    
plt.show(disp, "test viz")
ViewInteractiveWidget(plt.window)

~~~~ iteration  0 ~~~~~~~~~~~
m_hat =  [0. 0. 0. 0. 0. 0.]
(16564, 3)
residual (66256,)
~~~~ iteration  1 ~~~~~~~~~~~
m_hat =  [-0.34599979 -0.45758962  0.00315352 -0.00617179 -0.01010173 -0.39338193]
(16564, 3)
residual (66256,)
~~~~ iteration  2 ~~~~~~~~~~~
m_hat =  [-5.21549169e-02 -1.13094770e+00  7.33291572e-04 -1.56620805e-03
 -1.44139027e-02 -6.04915934e-01]
(16564, 3)
residual (66256,)
~~~~ iteration  3 ~~~~~~~~~~~
m_hat =  [ 0.49513983 -1.56394438  0.00167887  0.00308795 -0.01433306 -0.7192103 ]
(16564, 3)
residual (66256,)
~~~~ iteration  4 ~~~~~~~~~~~
m_hat =  [ 1.07761083 -1.76336648  0.00527027  0.00596725 -0.01239772 -0.78969595]
(16564, 3)
residual (66256,)
~~~~ iteration  5 ~~~~~~~~~~~
m_hat =  [ 1.58916269 -1.80149588  0.00949982  0.00733402 -0.00997579 -0.83877971]
(16564, 3)
residual (66256,)
~~~~ iteration  6 ~~~~~~~~~~~
m_hat =  [ 1.99732988 -1.74510936  0.01309584  0.0076637  -0.00766931 -0.87570496]
(16564, 3)
residual (66256,)
~~~~ iteration  7 ~~~~~~~~~~~
m_hat

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

# Test using spherical voxels to form distributions

In [21]:
#DEBUG: convert <apply_motion_profile> to linalg 
pc1 = new_cloud
pc2 = old_cloud
## only take first part of cloud
# pc1 = new_cloud[:5000]
# pc2 = old_cloud[:5000]
# # remove ground plane
# pc1 = new_cloud[new_cloud[:,2] > -1]
# pc2 = old_cloud[old_cloud[:,2] > -1]

# m_hat = np.array([3, -1, 0, 0, 0, -1])
# pc1 = new_cloud + 0.01*np.random.randn(np.shape(new_cloud)[0],3)
# m_hat = np.array([0, 0, 0, 0.2, 0, 0])
# pc2 = dc.apply_motion_profile(old_cloud, m_hat)

plt2 = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(pc1,  c = 'red'))
disp.append(Points(pc2, c = 'blue'))
plt2.show(disp, "test viz")
ViewInteractiveWidget(plt2.window)

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [22]:
from linear_corrector import LC

m_hat0 = np.array([0.,0, 0, 0, 0, 0]) #cold start
dc = LC(cloud1 = pc1, cloud2 = pc2, fid = 50, niter = 100, draw = True, m_hat0 = m_hat0)
ViewInteractiveWidget(dc.plt.window)

2023-04-24 16:03:20.690794: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-24 16:03:21.351053: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


~~~~~~~~~~~Iteration  0 ~~~~~~~~~~
took 0.017524242401123047 sec  to apply motion profile
took 0.001911163330078125 sec to get H
m_hat:  [ 4.44682874e-02 -1.04232073e-01  4.63253746e-05 -2.61299556e-05
 -9.31913164e-05 -6.16311133e-02]
~~~~~~~~~~~Iteration  1 ~~~~~~~~~~
took 0.0160977840423584 sec  to apply motion profile
took 0.0009083747863769531 sec to get H
m_hat:  [ 8.19167197e-02 -2.32608661e-01  1.43964513e-04 -3.37807069e-05
 -9.96433664e-05 -1.35305136e-01]
~~~~~~~~~~~Iteration  2 ~~~~~~~~~~


2023-04-24 16:03:23.758694: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x1b777cf0


took 0.016301631927490234 sec  to apply motion profile
took 0.0009205341339111328 sec to get H
m_hat:  [ 1.10743895e-01 -3.16339612e-01  2.66966439e-04 -5.49429096e-05
 -1.06059015e-04 -2.19950780e-01]
~~~~~~~~~~~Iteration  3 ~~~~~~~~~~
took 0.014546871185302734 sec  to apply motion profile
took 0.0009338855743408203 sec to get H
m_hat:  [ 1.34107977e-01 -4.41071510e-01  2.50029494e-04 -9.12932737e-05
 -8.97357313e-05 -3.08499575e-01]
~~~~~~~~~~~Iteration  4 ~~~~~~~~~~
took 0.01458883285522461 sec  to apply motion profile
took 0.0008883476257324219 sec to get H
m_hat:  [ 1.55983120e-01 -5.66649556e-01  2.00762996e-04 -1.15284463e-04
 -8.21810681e-05 -3.96297574e-01]
~~~~~~~~~~~Iteration  5 ~~~~~~~~~~
took 0.01629471778869629 sec  to apply motion profile
took 0.0009477138519287109 sec to get H
m_hat:  [ 1.71856523e-01 -6.22838497e-01  1.61595322e-04 -1.41147844e-04
 -6.71855087e-05 -4.89444554e-01]
~~~~~~~~~~~Iteration  6 ~~~~~~~~~~
took 0.014793634414672852 sec  to apply motion profile

took 0.002457141876220703 sec to get H
m_hat:  [ 2.99099731e+00 -1.02470827e+00 -4.58839670e-04 -4.83867825e-06
  1.16154690e-04 -9.98505950e-01]
~~~~~~~~~~~Iteration  39 ~~~~~~~~~~
took 0.016656160354614258 sec  to apply motion profile
took 0.0013196468353271484 sec to get H
m_hat:  [ 2.99373341e+00 -1.01866817e+00 -4.14657174e-04  3.26355439e-06
  1.13742994e-04 -9.99020696e-01]
~~~~~~~~~~~Iteration  40 ~~~~~~~~~~
took 0.016646146774291992 sec  to apply motion profile
took 0.0012722015380859375 sec to get H
m_hat:  [ 2.99576688e+00 -1.01393282e+00 -3.73758259e-04  9.05929573e-06
  1.10743844e-04 -9.99397218e-01]
~~~~~~~~~~~Iteration  41 ~~~~~~~~~~
took 0.01680588722229004 sec  to apply motion profile
took 0.0013184547424316406 sec to get H
m_hat:  [ 2.99721599e+00 -1.01024711e+00 -3.37470527e-04  1.35959608e-05
  1.07849264e-04 -9.99671876e-01]
~~~~~~~~~~~Iteration  42 ~~~~~~~~~~
took 0.01584172248840332 sec  to apply motion profile
took 0.0012574195861816406 sec to get H
m_hat:  [ 2

took 0.017991304397583008 sec  to apply motion profile
took 0.0013055801391601562 sec to get H
m_hat:  [ 2.99968195e+00 -1.00002718e+00 -1.83022639e-04  3.47405403e-05
  9.20878083e-05 -1.00015736e+00]
~~~~~~~~~~~Iteration  75 ~~~~~~~~~~
took 0.01745891571044922 sec  to apply motion profile
took 0.0012392997741699219 sec to get H
m_hat:  [ 2.99968123e+00 -1.00002813e+00 -1.83050724e-04  3.47504974e-05
  9.20770690e-05 -1.00015736e+00]
~~~~~~~~~~~Iteration  76 ~~~~~~~~~~
took 0.014018774032592773 sec  to apply motion profile
took 0.0027823448181152344 sec to get H
m_hat:  [ 2.99968123e+00 -1.00002849e+00 -1.82992153e-04  3.47559908e-05
  9.20762104e-05 -1.00015736e+00]
~~~~~~~~~~~Iteration  77 ~~~~~~~~~~
took 0.01638960838317871 sec  to apply motion profile
took 0.0012521743774414062 sec to get H
m_hat:  [ 2.99968123e+00 -1.00002873e+00 -1.82936608e-04  3.47552850e-05
  9.20814200e-05 -1.00015724e+00]
~~~~~~~~~~~Iteration  78 ~~~~~~~~~~
took 0.014934778213500977 sec  to apply motion pro

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [23]:
from numpy import sin, cos

M = dc.motion_profile

x = M[:,0]
y = M[:,1]
z = M[:,2]
phi = M[:,3]
theta = M[:,4]
psi = M[:,5]

T_rect_numpy = np.array([[cos(psi)*cos(theta), sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi), sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi), -x], [sin(psi)*cos(theta), sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi), -sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi), -y], [-sin(theta), sin(phi)*cos(theta), cos(phi)*cos(theta), -z], [np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), np.ones(len(x))]])
T_rect_numpy = np.transpose(T_rect_numpy, (2,0,1))
# print(T_rect_numpy)
print(np.shape(T_rect_numpy))

pc = np.append(old_cloud, np.ones([len(old_cloud),1]), axis=1)
print(np.shape(pc))

# T_rect_numpy = np.reshape(T_rect_numpy, (-1, 4))
# print(T_rect_numpy)
# print(np.shape(T_rect_numpy))

# ans =  T_rect_numpy @ pc[:,:,None]
# print(np.shape(ans[:,:,0]))

(11092, 4, 4)
(16564, 4)


# Try distortion correction on Newer College Dataset

###  As predicted, algorithm is having a trouble with cases where large swaths of the scene are missing points

In [24]:
#load data
idx = 140 #245
fn1 = "/media/derm/06EF-127D3/Newer College Dataset/06_Dynamic_Spinning/point_clouds/frame_" + str(idx) + ".npy"
pc1 = np.load(fn1)
fn2 = "/media/derm/06EF-127D3/Newer College Dataset/06_Dynamic_Spinning/point_clouds/frame_" + str(idx + 1) + ".npy"
pc2 = np.load(fn2)

#run ICET to get initial transforms of point clouds
it = ICET(cloud1 = pc1, cloud2 = pc2, fid = 50, niter = 10, 
           draw = True, group = 2, RM = False, DNN_filter = False)#, x0 = initial_guess)
print("\n predicted standard deviations of error: \n", it.pred_stds)
ViewInteractiveWidget(it.plt.window)

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9

 estimated solution vector X: 
 tf.Tensor([-0.00971789  0.04356265  0.00885437 -0.08903319 -0.03281014 -0.01775746], shape=(6,), dtype=float32)

 predicted standard deviations of error: 
 tf.Tensor(
[4.1027903e-04 3.1759459e-04 1.0126334e-04 1.6650010e-05 1.8151830e-05
 5.2045714e-05], shape=(6,), dtype=float32)


ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [62]:
#apply output of ICET to raw point clouds
trans = it.X[:3].numpy()
rot = R_tf(-it.X[3:]).numpy().T
pc2_transformed =  (pc2 @ rot) + trans 

In [61]:
# #for debug: inject exaggerated distortion to one of the clouds
m_hat_distort = np.array([0., 0.,0., 0., 0., 0.])
pc2_transformed = lc(pc2_transformed, m_hat)

plt3 = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(pc1,  c = 'red'))
disp.append(Points(pc2_transformed, c = 'blue'))
plt3.show(disp, "test viz")
ViewInteractiveWidget(plt3.window)

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [64]:
m_hat0 = np.array([0.,0, 0, 0, 0, 0])
dc = LC(cloud1 = pc1, cloud2 = pc2_transformed, fid = 50, niter = 100, draw = True, m_hat0 = m_hat0)
ViewInteractiveWidget(dc.plt.window)

~~~~~~~~~~~Iteration  0 ~~~~~~~~~~
took 0.02480030059814453 sec  to apply motion profile
took 0.0015740394592285156 sec to get H
m_hat:  [ 2.86304182e-03  2.10125372e-03 -4.39791474e-05  2.87483737e-04
  3.85768362e-05  7.28669576e-04]
~~~~~~~~~~~Iteration  1 ~~~~~~~~~~
took 0.02889251708984375 sec  to apply motion profile
took 0.003721475601196289 sec to get H
m_hat:  [ 4.89372015e-03  3.45721282e-03 -3.08639661e-04  4.13063332e-04
  7.97184766e-05  1.22703263e-03]
~~~~~~~~~~~Iteration  2 ~~~~~~~~~~
took 0.022322416305541992 sec  to apply motion profile
took 0.0016701221466064453 sec to get H
m_hat:  [ 0.00633206  0.00451085 -0.00065847  0.00047325  0.00012232  0.0015757 ]
~~~~~~~~~~~Iteration  3 ~~~~~~~~~~
took 0.02230691909790039 sec  to apply motion profile
took 0.003983974456787109 sec to get H
m_hat:  [ 0.00721026  0.0054062  -0.00091997  0.00051934  0.00015037  0.00180432]
~~~~~~~~~~~Iteration  4 ~~~~~~~~~~
took 0.028237342834472656 sec  to apply motion profile
took 0.0027759075

took 0.02525639533996582 sec  to apply motion profile
took 0.0016298294067382812 sec to get H
m_hat:  [ 0.00898154  0.00769892 -0.00183608  0.00064875  0.00025556  0.00224891]
~~~~~~~~~~~Iteration  40 ~~~~~~~~~~
took 0.0239260196685791 sec  to apply motion profile
took 0.002301454544067383 sec to get H
m_hat:  [ 0.00897983  0.0076988  -0.00183611  0.00064877  0.00025555  0.00224866]
~~~~~~~~~~~Iteration  41 ~~~~~~~~~~
took 0.02740168571472168 sec  to apply motion profile
took 0.0034880638122558594 sec to get H
m_hat:  [ 0.00897863  0.00769921 -0.00183619  0.00064878  0.00025556  0.00224868]
~~~~~~~~~~~Iteration  42 ~~~~~~~~~~
took 0.025289535522460938 sec  to apply motion profile
took 0.002627849578857422 sec to get H
m_hat:  [ 0.00897869  0.0076989  -0.00183615  0.00064879  0.00025557  0.00224871]
~~~~~~~~~~~Iteration  43 ~~~~~~~~~~
took 0.023635149002075195 sec  to apply motion profile
took 0.0017833709716796875 sec to get H
m_hat:  [ 0.00897786  0.00769929 -0.00183627  0.00064878  0

took 0.0021209716796875 sec to get H
m_hat:  [ 0.00897889  0.00769929 -0.00183623  0.00064881  0.00025562  0.00224883]
~~~~~~~~~~~Iteration  79 ~~~~~~~~~~
took 0.020917177200317383 sec  to apply motion profile
took 0.0016918182373046875 sec to get H
m_hat:  [ 0.00897711  0.00769944 -0.00183628  0.0006488   0.00025559  0.00224854]
~~~~~~~~~~~Iteration  80 ~~~~~~~~~~
took 0.02335333824157715 sec  to apply motion profile
took 0.0016598701477050781 sec to get H
m_hat:  [ 0.00897837  0.00769954 -0.00183637  0.00064881  0.00025558  0.00224866]
~~~~~~~~~~~Iteration  81 ~~~~~~~~~~
took 0.02396535873413086 sec  to apply motion profile
took 0.0022416114807128906 sec to get H
m_hat:  [ 0.00897964  0.00769999 -0.00183645  0.0006488   0.00025561  0.00224894]
~~~~~~~~~~~Iteration  82 ~~~~~~~~~~
took 0.02285456657409668 sec  to apply motion profile
took 0.0015823841094970703 sec to get H
m_hat:  [ 0.00897918  0.00770005 -0.00183638  0.00064882  0.00025561  0.00224897]
~~~~~~~~~~~Iteration  83 ~~~~~~~

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)